In [7]:
#Mount Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Required Libraries
import cv2   #video  capture
import os


In [ ]:
#Define Path
video_path = '/content/drive/MyDrive/DataSet_TeleICU.mp4'
save_dir = '/content/drive/MyDrive/Colab Notebooks/save_dir'

# Create directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


In [ ]:
# Read Video and Convert to Images Alternately
def video_to_images(video_path, save_dir, interval=2):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_image_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Save every nth frame
        if frame_count % interval == 0:
            image_path = os.path.join(save_dir, f'image_{saved_image_count:05d}.jpg')
            cv2.imwrite(image_path, frame)
            saved_image_count += 1
        frame_count += 1

    cap.release()
    print(f'Total {saved_image_count} images saved.')
# Call the function to save every 2th frame as an example
video_to_images(video_path, save_dir, interval=2)



Total 8384 images saved.


In [ ]:
# Performing k-means Clustering and Saving Segmented Images
import cv2
import numpy as np
import os

# Function to perform k-means clustering
def perform_kmeans_clustering(image_path, k=3):
    # Read image
    img = cv2.imread(image_path)
    # Convert image to RGB (if it's in BGR format)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Reshape image to a 2D array of pixels (height * width, 3)
    pixels = img_rgb.reshape((-1, 3))
    pixels = np.float32(pixels)

    # Define criteria and apply k-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert centers to uint8
    centers = np.uint8(centers)

    # Reshape labels to match the shape of original image
    labels = labels.reshape(img.shape[:2])

    # Create segmented image based on k-means clustering
    segmented_image = centers[labels]

    # Reshape segmented image to match original image shape
    segmented_image = segmented_image.reshape(img.shape)

    return segmented_image

# Path to your directory containing images
save_dir = '/content/drive/MyDrive/Colab Notebooks/save_dir'

# Directory to save segmented images
segmented_dir = '/content/drive/MyDrive/Colab Notebooks/segmented_dir'

# Create the segmented directory if it doesn't exist
os.makedirs(segmented_dir, exist_ok=True)

# List all image files in save_dir
image_files = os.listdir(save_dir)

# Process each image
for img_file in image_files:
    img_path = os.path.join(save_dir, img_file)
    segmented_img = perform_kmeans_clustering(img_path)

    # Save segmented image
    segmented_img_path = os.path.join(segmented_dir, f'segmented_{img_file}')
    cv2.imwrite(segmented_img_path, segmented_img)

print(f'Segmented images saved to {segmented_dir}')

In [ ]:
#Model1:-
#Splitting the Single Model into 3 Models
!pip install ultralytics
# Train TOLOv8 Model
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # Use a lightweight model for faster inference

# Train the model
model.train(data='coco128.yaml', epochs=50)  # Replace 'coco128.yaml' with your dataset YAML file

In [ ]:
# Validate and Test the Model
# Validate the model
model.val()

# Test the model
results = model.predict('/path/to/test/images', save=True)


In [ ]:
# Model2:- Detect if Patient is Alone or with Family Members
# Assume results from Model 1 are saved
model1_results = results  # Replace with actual results

# Check if patient is alone or with family members
alone = False
with_family = False

for result in model1_results:
    if 'Patient' in result and 'Family Members' in result:
        with_family = True
    elif 'Patient' in result:
        alone = True

if alone or with_family:
    proceed_to_model3 = True
else:
    proceed_to_model3 = False


In [ ]:
# Model3:- Detect Patient Movement
#Train YOLOv8 Model for Movement Detection
# Define custom dataset YAML file
movement_data = '''
path: /path/to/dataset
train: images/train
val: images/val
test: images/test

nc: 4  # Number of classes
names: ['Breathlessness', 'HandLegMovement', 'Fall', 'Wakeup']
'''

with open('/content/movement_data.yaml', 'w') as f:
    f.write(movement_data)

# Train the model
model_movement = YOLO('yolov8n.pt')
model_movement.train(data='/content/movement_data.yaml', epochs=50)

In [ ]:
# Validate and Test the Movement Detection Model
# Validate the model
model_movement.val()

# Test the model
movement_results = model_movement.predict('/path/to/test/images', save=True)


In [ ]:
#Alert System
#Check Movements and Send Alert
alert = False
for result in movement_results:
    if 'Breathlessness' in result or 'Fall' in result:
        alert = True
        break

if alert:
    # Send alert message
    print("Alert: Immediate attention needed in TeleICU")


In [ ]:
#Calculating Accuracy and Performance Metrics
#Calculate Performance Metrics for YOLOv8
# Training and Validation
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # Use a lightweight model for faster inference

# Train the model
model.train(data='coco128.yaml', epochs=50)  # Replace 'coco128.yaml' with your dataset YAML file


In [ ]:
#Validation and Metrics Calculation
# Validate the model
metrics = model.val()

# Print the metrics
print(metrics)

In [ ]:
metrics = model.val()

print("Precision: ", metrics['precision'])
print("Recall: ", metrics['recall'])
print("mAP@0.5: ", metrics['map50'])
print("mAP@0.5:0.95: ", metrics['map'])

In [ ]:
#Accuracy
# Predict on the Validation Set
results = model.predict('/path/to/validation/images', save=True)

#Compare Predictions with Ground Truth
# Load ground truth annotations
with open('/path/to/ground_truth/annotations.json') as f:
    ground_truth = json.load(f)

# Compare predictions with ground truth
correct_predictions = 0
total_predictions = len(results)

for result, gt in zip(results, ground_truth['annotations']):
    # Assuming gt and result have the same format
    if result['category_id'] == gt['category_id']:
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print("Accuracy: ", accuracy)
